In [1]:
from haystack.components.embedders import OpenAIDocumentEmbedder, OpenAITextEmbedder
from haystack_integrations.document_stores.weaviate.document_store import WeaviateDocumentStore
from haystack_integrations.components.retrievers.weaviate.embedding_retriever import WeaviateEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack_integrations.components.retrievers.elasticsearch import ElasticsearchBM25Retriever


embedder = OpenAIDocumentEmbedder(model="text-embedding-3-small")
text_embedder = OpenAITextEmbedder(model="text-embedding-3-small")
weaviate_store = WeaviateDocumentStore(url="http://localhost:8088")
elasticsearch_store = ElasticsearchDocumentStore(hosts= "http://localhost:9200")
weaviate_retriever = WeaviateEmbeddingRetriever(document_store=weaviate_store, top_k=3)
elasticsearch_retriever = ElasticsearchBM25Retriever(document_store=elasticsearch_store, top_k=3)

template = """
Answer the question only using the following context. Do not use any external information. 
If the answer is not present in the context, please answer with "I don't know".

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder = PromptBuilder(template=template)

generator = OpenAIGenerator(model="gpt-4o-mini")

/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.0 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(
/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/haystack/components/generators/openai.py:106: DeprecationWarning: In the upcoming releases 'gpt-3.5-turbo' will be replaced by 'gpt-4o-mini' as the default model
  warnings.warn(


### Joiner

In [2]:
from haystack import Pipeline
from haystack.components.joiners.document_joiner import DocumentJoiner

joiner = DocumentJoiner(join_mode="reciprocal_rank_fusion", top_k=3)

p = Pipeline()

p.add_component("text_embedder", text_embedder)
p.add_component("weaviate_retriever", weaviate_retriever)
p.add_component("elasticsearch_retriever", elasticsearch_retriever)
p.add_component("joiner", joiner)

p.connect("text_embedder.embedding", "weaviate_retriever.query_embedding")
p.connect("weaviate_retriever", "joiner")
p.connect("elasticsearch_retriever", "joiner")

🚅 Components
  - text_embedder: OpenAITextEmbedder
  - weaviate_retriever: WeaviateEmbeddingRetriever
  - elasticsearch_retriever: ElasticsearchBM25Retriever
  - joiner: DocumentJoiner
🛤️ Connections
  - text_embedder.embedding -> weaviate_retriever.query_embedding (List[float])
  - weaviate_retriever.documents -> joiner.documents (List[Document])
  - elasticsearch_retriever.documents -> joiner.documents (List[Document])

In [3]:
query = "How many dinosaur species existed before the extinction event?"

result = p.run(data={"elasticsearch_retriever": {"query": query}, 
            "text_embedder": {"text": query}}, include_outputs_from={"weaviate_retriever", "elasticsearch_retriever"})

result

/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/posthog/client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/posthog/request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


{'text_embedder': {'meta': {'model': 'text-embedding-3-small',
   'usage': {'prompt_tokens': 10, 'total_tokens': 10}}},
 'joiner': {'documents': [Document(id=433ad671-ea22-4ae1-9bc4-1cce06a0e6ee, content: 'Just before the K-Pg extinction event, the number of non-avian dinosaur species that existed globall...', meta: {'h3': 'Pre-extinction diversity', 'source_id': '93000a3fb02b99d2d115cd4042256d2f5db2a0ff3928927ca14465276534a75e', 'split_id': 235.0, 'file_path': 'Dinosaur.html', 'title': 'Dinosaurs', 'h4': None, 'h2': 'Extinction of major groups'}, score: 1.0, embedding: vector of size 1536),
   Document(id=3f45e988-b093-4d9c-bde1-67e2e4c3b599, content: 'All non-avian dinosaurs and most lineages of birds became extinct in a mass extinction event, called...', meta: {'file_path': 'Dinosaur.html', 'source_id': '93000a3fb02b99d2d115cd4042256d2f5db2a0ff3928927ca14465276534a75e', 'split_id': 234, 'title': 'Dinosaurs', 'h2': 'Extinction of major groups'}, score: 0.4919354838709677, embedding: 

Skipping LLM calls...straightforward

### Explore graph calling

In [4]:
query = "Tell me in short about the physiology of dinosaurs."

result = p.run(data={"elasticsearch_retriever": {"query": query}, 
            "text_embedder": {"text": query}}, include_outputs_from={"weaviate_retriever", "elasticsearch_retriever"})

result

/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/posthog/client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/posthog/request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


{'text_embedder': {'meta': {'model': 'text-embedding-3-small',
   'usage': {'prompt_tokens': 10, 'total_tokens': 10}}},
 'joiner': {'documents': [Document(id=8a7f94b9-3ae4-45a8-90ba-c9f5377c4b5b, content: 'World War II caused a pause in palaeontological research; after the war, research attention was also...', meta: {'h3': '"Dinosaur renaissance" and beyond', 'split_id': 33.0, 'source_id': '93000a3fb02b99d2d115cd4042256d2f5db2a0ff3928927ca14465276534a75e', 'file_path': 'Dinosaur.html', 'title': 'Dinosaurs', 'h4': None, 'h2': 'History of study'}, score: 0.9760624679979518, embedding: vector of size 1536),
   Document(id=3aacf659-c395-452b-92a1-e2e1d2b81ec3, content: 'The popular preoccupation with dinosaurs has ensured their appearance in literature, film, and other...', meta: {'file_path': 'Dinosaur.html', 'source_id': '93000a3fb02b99d2d115cd4042256d2f5db2a0ff3928927ca14465276534a75e', 'split_id': 244, 'title': 'Dinosaurs', 'h2': 'Cultural depictions'}, score: 0.5, embedding: vector of

One approach
- Loop over all documents returned by Joiner and anchor on corresponding chunks in Neo4j graph
- Find parent of chunk
- Spread out 2 levels to find all non-chunk nodes
- Compare cosine similarity with question???
- Find the most matching node and return all chunks...

Another approach
- If LLM finds that context is not enough to answer the question, it should ask for more context
- For each retrieved chunk, find title node, create page hierarchy from graph using title node and provide the page hierarchy to the LLM
- Let LLM decide the deepest level in the hierarchy which it feels can sufficiently answer the question. Provide all chunks for that deepest level as context for the LLM to answer.

### Construct hierarchy of page given a chunk id